In [1]:
import pandas as pd
import numpy as np

In [2]:
iris = pd.read_csv("iris.tmls")
irisValueTypes = iris.iloc[0].tolist()
iris = iris.drop([0, 0])

wine = pd.read_csv("wine.tmls")
wineValueTypes = wine.iloc[0].tolist()
wine.drop([0, 0])

tictactoe = pd.read_csv("tictactoe.tmls")
tictactoeValueTypes = tictactoe.iloc[0].tolist()
tictactoe = tictactoe.drop([0, 0])

dataframes=[iris, wine, tictactoe]
valueTypes = [irisValueTypes, wineValueTypes, tictactoeValueTypes]

# NODE 

In [3]:
class Node():
    def __init__(self, featureIndex = None, threshold = None, leftChild = None, rightChild = None, informationGain = None, noOfSamples = None, value = None):
        #Decision node
        self.featureIndex = featureIndex
        self.threshold = threshold
        self.leftChild = leftChild
        self.rightChild = rightChild
        self.informationGain = informationGain
        self.noOfSamples = noOfSamples
        
        #Leaf Node
        self.value = value

# TREE

In [4]:
class DecisionTree():
    def __init__(self, maxDepth = 10, minSamples = 1, features = None, valueTypes = None):
        self.root = None

        self.maxDepth = maxDepth
        self.minSamples = minSamples
        self.features = features
        self.valueTypes = valueTypes


    def fit(self, trainDf):
        self.root = self.growTree(trainDf = trainDf)


    def growTree(self, currentDepth = 0, trainDf = None):
        noOfSamples = trainDf.shape[0]
        
        if(noOfSamples < self.minSamples or currentDepth > self.maxDepth or self.isHomogeneous(trainDf)):
            #Create Leaf Node
            leafValue = self.getLeafValue(trainDf.iloc[:,-1])
            return Node(noOfSamples = trainDf.shape[0], value = leafValue)
        
        #Get the best split for the current node
        bestSplit = self.getBestSplit(noOfSamples, trainDf)

        #Don't continue if information gain is below zero
        if(bestSplit["informationGain"] >= 0):

            #Continue building on the left and the right subtree
            leftSubtree = self.growTree(currentDepth + 1, bestSplit["leftDf"])
            rightSubtree = self.growTree(currentDepth + 1, bestSplit["rightDf"])

            #Decision node
            return Node(bestSplit["featureIndex"], bestSplit["threshold"], leftSubtree, rightSubtree, bestSplit["informationGain"], trainDf.shape[0])


    def getBestSplit(self, noOfSamples, trainDf):
        #initialize best split
        bestSplit = {}

        maxInformationGain = -float("inf")
        
        #Iterate each feature of the training dataset
        for featureIndex in range(len(self.features)):
            #Select all values from the given feature
            featureValues = trainDf.iloc[:, featureIndex]
            
            #Select only unique values in the given feature values
            candidateThresholds = featureValues.unique()
            
            #Iterate each unique column value to find the best splitting criteria
            for threshold in candidateThresholds:
                #Get the potential child nodes for the new split
                leftSplit, rightSplit = self.getSplit(featureIndex, threshold, trainDf)

                #Check so both child nodes are populated
                if(leftSplit.shape[0] > 0 and rightSplit.shape[0] > 0):
                    #Get target from: current node, left node, and right node
                    currentTarget = trainDf.iloc[:, -1]
                    leftTarget = leftSplit.iloc[:, -1]
                    rightTarget = rightSplit.iloc[:, -1]

                    #Calculate the information gain with the new child nodes
                    currentInformationGain = self.getInformationGain(currentTarget, leftTarget, rightTarget)

                    #Check if the currently calculated information gain is greather than the previous best information gain
                    if(currentInformationGain > maxInformationGain):
                        #Update best information gain
                        maxInformationGain = currentInformationGain

                        #Update the best split with current information
                        bestSplit["featureIndex"] = featureIndex
                        bestSplit["threshold"] = threshold
                        bestSplit["leftDf"] = leftSplit
                        bestSplit["rightDf"] = rightSplit
                        bestSplit["informationGain"] = maxInformationGain

        #Return the best split found
        return bestSplit

                    
    def getSplit(self, featureIndex, threshold, trainDf):
        leftSplitList = []
        rightSplitList = []

        #Convert to dictionary for faster iteration speed
        trainDict = trainDf.to_dict("records")

        if(self.valueTypes[featureIndex] == "n"):
            #NOMINAL(n) SPLIT
            for trainRow in trainDict:
                if(trainRow[self.features[featureIndex]] == threshold):
                    leftSplitList.append(trainRow)
                else:
                    rightSplitList.append(trainRow)

            leftSplit_n = pd.DataFrame(leftSplitList)
            rightSplit_n = pd.DataFrame(rightSplitList)
            return leftSplit_n, rightSplit_n
        else:
            #NUMERIC(r) SPLIT
            for trainRow in trainDict:
                if(trainRow[self.features[featureIndex]] <= threshold):
                    leftSplitList.append(trainRow)
                else:
                    rightSplitList.append(trainRow)

            leftSplit_r = pd.DataFrame(leftSplitList)
            rightSplit_r = pd.DataFrame(rightSplitList)
            return leftSplit_r, rightSplit_r
        

    def getInformationGain(self, current, leftChild, rightChild):
        #Get weight for each child node
        leftWeight = leftChild.shape[0] / current.shape[0]
        rightWeight = rightChild.shape[0] / current.shape[0]

        #Calculate information gain with the help of Gini Impurity
        return self.giniImpurity(current) - (leftWeight * self.giniImpurity(leftChild) + rightWeight * self.giniImpurity(rightChild))


    #Calculates the Gini Impurity
    def giniImpurity(self, targets):
        uniqueLabels = targets.unique()
        giniScore = 0

        for label in uniqueLabels:
            labelProbability = len(targets[targets == label]) / len(targets)
            giniScore += labelProbability**2

        return 1 - giniScore

    def getLeafValue(self, targets):
        targets = targets.tolist()
        return max(targets, key=targets.count)

    
    def isHomogeneous(self, trainDf):
        #Extract all targets into list
        targets = trainDf.iloc[:, -1].tolist()

        #Get each unique label to test if homogeneous
        uniqueLabels = set(targets)

        #Check if a sample in the tree's target is greater than 95%, if 95% or higher it is considered homogeneous
        for label in uniqueLabels:
            if targets.count(label) / len(targets) >= 0.95:
                return True
        return False


    #Way to visualize tree if wanted
    def drawTree(self, tree = None, indent = "", nodeDirection="root"):
        if tree == None:
            return

        if tree.value == None:
            print(indent, nodeDirection, tree.threshold, tree.informationGain, self.features[tree.featureIndex], tree.noOfSamples)
            self.drawTree(tree = tree.leftChild, indent = indent + "   ", nodeDirection = "left")
            self.drawTree(tree = tree.rightChild, indent = indent + "   ", nodeDirection = "right")
        else:
            print(indent, nodeDirection, "LEAF", tree.value, tree.noOfSamples)

    def getPredictions(self, testDf):
        testDict = testDf.to_dict("records")
        predictions = []

        for testRow in testDict:
            predictions.append(self.getPrediction(testRow, self.root))
        return predictions
        


    def getPrediction(self, testRow, tree):
        #Check if leaf node
        if tree.leftChild == None or tree.rightChild == None:
            return tree.value
        
        featureValue = testRow[self.features[tree.featureIndex]]

        #Nominal predictor
        if self.valueTypes[tree.featureIndex] == "n":
            if featureValue == tree.threshold:
                return self.getPrediction(testRow, tree.leftChild)
            else:
                return self.getPrediction(testRow, tree.rightChild)
        #Numeric predictor
        else:
            if featureValue <= tree.threshold:
                return self.getPrediction(testRow, tree.leftChild)
            else:
                return self.getPrediction(testRow, tree.rightChild)

# PREP

In [5]:
#Data prepartion and preprocessing
def partitionData(dataframe, dataframeTypes):
    #Shuffle dataframe
    dataframe = dataframe.sample(frac=1)

    #Split 80% to train-dataframe and rest to test-dataframe
    trainDf = dataframe.sample(frac=0.8)
    testDf = dataframe.drop(trainDf.index)

    #Reset Indexes
    trainDf.reset_index(drop=True)
    testDf.reset_index(drop=True)

    return trainDf, testDf

# LAUNCH

In [6]:
#trainDf, testDf = partitionData(tictactoe, tictactoeValueTypes)
trainDf, testDf = partitionData(iris, irisValueTypes)
#trainDf, testDf = partitionData(wine, wineValueTypes)

trainFeatures = trainDf.drop(["class"], axis = 1).columns.tolist()

#tree = DecisionTree(features = trainFeatures, valueTypes = tictactoeValueTypes)
tree = DecisionTree(features = trainFeatures, valueTypes = irisValueTypes)
#tree = DecisionTree(features = trainFeatures, valueTypes = wineValueTypes)

tree.fit(trainDf)
tree.drawTree(tree = tree.root)

 root 1.9 0.320035140562249 petal length 120
    left LEAF Iris-setosa 37
    right 4.7 0.3690774556647688 petal length 83
       left LEAF Iris-versicolor 39
       right 1.7 0.06634527089072523 petal width 44
          left 2.6 0.16666666666666669 sepal width 8
             left LEAF Iris-virginica 2
             right 5.0 0.2222222222222222 petal length 6
                left LEAF Iris-versicolor 3
                right 6.0 0.4444444444444444 sepal length 3
                   left LEAF Iris-versicolor 1
                   right LEAF Iris-virginica 2
          right LEAF Iris-virginica 36


In [8]:
predictions = tree.getPredictions(testDf)
testTargets = testDf.iloc[:, -1].tolist()

correct = 0
for i in range(len(predictions)):
    if(predictions[i] == testTargets[i]):
        correct += 1
print("Accuracy: " + str(correct/len(predictions)))

Accuracy: 0.9666666666666667
